### Github에서 source를 내려받고 directory를 이동합니다.

In [ ]:
!git clone https://github.com/scpark20/AI-for-Music.git
import os
os.chdir('AI-for-Music/source')

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [10]:
from easydict import EasyDict
from hparams import dataset_hparams

model_hparams = EasyDict(n_tokens = dataset_hparams.offsets.pedal_off,
                         embedding_dim = 512,
                         hidden_dim = 1024
                        )      

### Model을 구성합니다.
Model은 Embedding, RNN (LSTM), Linear Layer로 구성되어 있습니다.

In [19]:
class Model(nn.Module):
    def __init__(self, model_hparams):
        super().__init__()
        self.hp = model_hparams
        self.step = nn.Parameter(torch.zeros(1).long(), requires_grad=False)
        
        # Embdding Layer
        # token값을 vector로 변환합니다. 
        self.embedding = nn.Embedding(self.hp.n_tokens, self.hp.embedding_dim)
        
        # RNN Layer
        # 현재의 input vector와 이전 시점의 state vector를 입력 받아 현재 시점의 state vector를 출력합니다.
        self.rnn = nn.LSTM(input_size=self.hp.embedding_dim, hidden_size=self.hp.hidden_dim,
                        num_layers=3, batch_first=True, dropout=0.1)
        
        # Layer Layer
        # 현재 시점의 state vector를 입력 받아 미래 시점의 token에 대한 probability distribution을 출력합니다.
        self.out_layer = nn.Linear(self.hp.hidden_dim, self.hp.n_tokens)
        
    def forward(self, x):
        # x : (batch, length)
        
        # (batch, length, model_dim)
        x = self.embedding(x)
        # (batch, length, hidden_dim)
        x, _ = self.rnn(x)
        # (batch, length, n_tokens)
        x = self.out_layer(x)
        return x
    
    def _get_initial_state(self, batch_size):
        h = torch.zeros(3, batch_size, self.hp.hidden_dim)
        c = torch.zeros(3, batch_size, self.hp.hidden_dim)
        return (h, c)
    
    def inference(self, x, state=None, temperature=1.0):
        # x : (batch, length)
        
        # (batch, length, model_dim)
        x = self.embedding(x)
        # (batch, length, hidden_dim)
        x, state = self.rnn(x, state)
        # (batch, length, n_tokens)
        x = self.out_layer(x)
        # (batch, 1)
        x = torch.distributions.categorical.Categorical(logits=x[:, -1:]/temperature).sample()
        return x, state

### Model Init.

In [20]:
model = Model(model_hparams)
print(model)

Model(
  (embedding): Embedding(389, 512)
  (rnn): LSTM(512, 1024, num_layers=3, batch_first=True, dropout=0.1)
  (out_layer): Linear(in_features=1024, out_features=389, bias=True)
)


### Test for Training

In [21]:
batch = 2
length = 100
x = torch.randint(389, size=(batch, length))
y = model(x)
print(y.shape)

torch.Size([2, 100, 389])


### Test for Inference

In [24]:
state = model._get_initial_state(batch)
x = torch.randint(389, size=(batch, 1))
y, (h, c) = model.inference(x, state)
print(y.shape)
print(h.shape, c.shape)

torch.Size([2, 1])
torch.Size([3, 2, 1024]) torch.Size([3, 2, 1024])
